In [1]:
import os
import cv2
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [8]:
# Функция для обрезки изображения до нужной области
def crop_image(img, crop_region):
    x, y, w, h = crop_region
    return img[y:y+h, x:x+w]

In [9]:

# Загрузка изображений из директории с обрезкой
def load_images_from_directory(directory, crop_region, target_size=(128, 128)):
    images = []
    image_paths = []
    for filename in os.listdir(directory):
        if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".jpeg"):
            img_path = os.path.join(directory, filename)
            img = cv2.imread(img_path)
            if img is not None:
                # Обрезаем изображение до нужной области
                img_cropped = crop_image(img, crop_region)
                # Масштабируем изображение к нужному размеру
                img_resized = cv2.resize(img_cropped, target_size)
                img_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)  # Преобразуем в градации серого
                images.append(img_gray.flatten())  # Преобразуем изображение в одномерный массив
                image_paths.append(img_path)
    return np.array(images), image_paths

In [3]:
# Обучение начальной модели One-Class SVM на нормальных данных
def train_one_class_svm(normal_data):
    scaler = StandardScaler()
    normal_data_scaled = scaler.fit_transform(normal_data)

    # Обучение модели One-Class SVM
    model = OneClassSVM(gamma='auto', kernel='rbf', nu=0.1)
    model.fit(normal_data_scaled)

    return model, scaler

In [4]:
# Проверка изображения на аномалии
def is_anomaly(image, model, scaler):
    img_scaled = scaler.transform(image.reshape(1, -1))  # Масштабируем изображение
    prediction = model.predict(img_scaled)
    return prediction == -1  # -1 означает аномалию


In [5]:
# Функция для отображения изображения
def display_image(image_path):
    img = cv2.imread(image_path)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

In [6]:
# Активное обучение: пользователь помечает аномалии
def active_learning(anomaly_images, model, scaler):
    corrected_labels = []
    corrected_data = []
    
    for img_path, img in anomaly_images:
        print(f"Проверка изображения: {img_path}")
        display_image(img_path)
        
        # Пользовательская пометка
        label = input("Является ли изображение аномалией? (1 - да, 0 - нет): ")
        if label == '1':
            corrected_labels.append(1)  # Аномалия
        else:
            corrected_labels.append(0)  # Не аномалия
        corrected_data.append(img)
    
    return np.array(corrected_data), np.array(corrected_labels)

In [7]:
# Дообучение модели на новых данных с пользовательскими метками
def retrain_model(corrected_data, corrected_labels, model, scaler):
    # Преобразуем данные в векторы признаков и масштабируем
    corrected_data_scaled = scaler.transform(corrected_data)

    # Тренируем новую модель, используя корректированные данные
    kmeans = KMeans(n_clusters=2)  # Кластеризация на 2 кластера (нормальные и аномальные данные)
    kmeans.fit(corrected_data_scaled)

    return kmeans


In [10]:
# Основной код
if __name__ == "__main__":
    # Определяем регион обрезки: (x, y, width, height)
    crop_region = (1390, 180, 624, 512)

    # Путь к нормальным данным (правильные изображения)
    normal_data_dir = "object3_cut"
    
    # Путь к неправильным данным (для активного обучения)
    anomaly_data_dir = "object3_bad"

    # Шаг 1: Загрузка нормальных изображений и обучение модели One-Class SVM
    normal_images, _ = load_images_from_directory(normal_data_dir, crop_region)
    model, scaler = train_one_class_svm(normal_images)

    # Шаг 2: Проверка на неправильных изображениях
    anomaly_images, anomaly_image_paths = load_images_from_directory(anomaly_data_dir, crop_region)
    detected_anomalies = []

    for img, img_path in zip(anomaly_images, anomaly_image_paths):
        if is_anomaly(img, model, scaler):
            detected_anomalies.append((img_path, img))

    # Шаг 3: Активное обучение - пользователь помечает аномалии
    corrected_data, corrected_labels = active_learning(detected_anomalies, model, scaler)

    # Шаг 4: Дообучение модели с учётом пользовательских меток
    final_model = retrain_model(corrected_data, corrected_labels, model, scaler)

    print("Обучение завершено. Модель скорректирована с учётом пользовательских данных.")

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
